In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

# Go up one directory from `b/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()

DEFAULT_TRANSFORMERS_MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 16_000,
    "load_in_4bit": True,
    "enable_thinking": False,  # main_gpu interpretation
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **DEFAULT_TRANSFORMERS_MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

╔════════════════════════════════════════════════════════════════════╗
║                                                                    ║
║ ▓▓▓▓▓   ▓▓▓▓                                   ▓▓▓                 ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓                   ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓                  ║
║  ▓▓   ▓▓ ▓▓  ▓     ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓                   ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓                   ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready   ║
║ Faster builds, clearer flows, production-first                     ║
║                                                                    ║
║ Version: unknown | Python: 3.11.13                                 ║
║ OS: Linux 6.14.0-33-generic (x86_64)                               ║
║ Torch: 2.7.1+cu126   CUDA: yes (12.6)                              ║
║ MPS:

/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     | 2025-11-05 10:53:35 | config.py:<module> | PyTorch version 2.7.1+cu126 available.


/home/nomi/anaconda3/envs/LLMs/lib/python3.11/importlib/__init__.py:126: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-05 10:53:36 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-11-05 10:53:39 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-11-05 10:53:39 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.
INFO     | 2025-11-05 10:53:39 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]

INFO     | 2025-11-05 10:53:44 | transformers.py:init_model | Transformers model initialized successfully.


In [3]:
from IPython.display import display, Markdown, clear_output

system_prompt = "You are a joker."
user_prompt = """Joke about yourself."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    print(chunk, flush=True, end="")

Ah, you caught me! I'm not just a joker—I'm a *joker with a PhD in puns*! 😄 But don't worry, I'm not here to steal your thunder. I'm here to *steal your laughs*! 

So, here's my joke:  
**"I tried to tell a joke about myself, but it was a bit of a *loop*—I kept going around in circles, and I couldn't find the punchline!"**  

*Cracks a grin*  
Want to hear another one? Or should I just... *let the silence speak for itself*? 😏

## RAG wiring so the agent “understands” the Neurosurf codebase

You’ll ingest the repo once, then run a retriever to answer code questions. The Planner can call the retriever first to form a precise implementation plan.

### FileReader and Chunker Test

In [4]:
# scripts/index_repo_for_rag.py
from neurosurfer.rag.ingestor import RAGIngestor
from neurosurfer.rag.chunker import Chunker
from neurosurfer.rag.filereader import FileReader
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder

embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
vs = ChromaVectorStore(collection_name="neurosurf-repo")
ing = RAGIngestor(
    embedder=embedder, 
    vector_store=vs, 
    chunker=Chunker(), 
    file_reader=FileReader(),
    default_metadata={"collection": "neurosurf"}
)

ing.add_directory("/home/nomi/workspace/neurosurfer/neurosurfer")  # the repo root
print(ing.build())


INFO     | 2025-11-05 10:53:57 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-05 10:53:57 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-11-05 10:53:57 | posthog.py:__init__ | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[Init] ChromaVectorStore initialized with collection: neurosurf-repo


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.71it/s]


{'status': 'ok', 'sources': 96, 'chunks': 599, 'unique_chunks': 599, 'added': 599, 'finished_at': 1762325640.3801126}


## RAG AGENT

In [6]:
# scripts/rag_helper.py
from neurosurfer.agents.rag import RAGAgent, RAGAgentConfig
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
from neurosurfer.models.chat_models.openai import OpenAIModel

vs = ChromaVectorStore(collection_name="neurosurf-repo")
# embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
# llm = OpenAIModel(model_name="gpt-4o-mini")
rag_agent = RAGAgent(
    llm=LLM, 
    vectorstore=vs, 
    embedder=embedder, 
    config=RAGAgentConfig(top_k=6)
)

[Init] ChromaVectorStore initialized with collection: neurosurf-repo


Test RagAgent

In [8]:
from IPython.display import display, Markdown

# test retrieve
# retrieved_obj = rag_agent.retrieve("What is the main difference between transformers and unsloth?")
# print(retrieved_obj.context)

# test run
llm_response = rag_agent.run("Explain Chunker Configurations in table form.")
response = ""
md_display = display(Markdown(""), display_id=True)
for chunk in llm_response:
    response += chunk
    md_display.update(Markdown(response))


Here's a **table summarizing the key configurations of the `Chunker` class** from the provided code, including their purposes and default values:

| Configuration Parameter             | Description                                                                 | Default Value |
|------------------------------------|-----------------------------------------------------------------------------|---------------|
| `fallback_chunk_size`              | Number of lines per chunk when using line-based splitting.                 | 25            |
| `overlap_lines`                    | Number of overlapping lines between consecutive line-based chunks.         | 3             |
| `max_chunk_lines`                  | Safety cap on lines per chunk to prevent excessively large chunks.         | 1000          |
| `comment_block_threshold`          | Minimum number of consecutive comment-only lines to treat as a comment block. | 4             |
| `char_chunk_size`                  | Number of characters per chunk when using character-based splitting.       | 1000          |
| `char_overlap`                     | Number of overlapping characters between consecutive character-based chunks. | 150           |
| `readme_max_lines`                 | Maximum number of lines allowed for README files.                           | 30            |

### Notes:
- These configurations are defined in the `ChunkerConfig` class.
- They control how the `Chunker` splits text into chunks, with options for **line-based** (code-friendly) and **character-based** (prose-friendly) chunking.
- The `Chunker` uses these settings to determine chunk size, overlap, and behavior for different file types.

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.02it/s]

INFO     | 2025-11-05 10:58:42 | agent.py:retrieve | [RAGRetriever] Retrieved 6 chunks


## Tools Router Agent Test

In [34]:
from neurosurfer.agents.tools_router_agent import ToolsRouterAgent
from neurosurfer.tools.toolkit import Toolkit

from neurosurfer.tools.tool_spec import ToolSpec, ToolParam, ToolReturn
from neurosurfer.tools.base_tool import BaseTool, ToolResponse
from neurosurfer.tools.common.general_query_assistant import GeneralQueryAssistantTool

# Simple Calculator Tool
class CalculatorTool(BaseTool):
    spec = ToolSpec(
        name="calculator",
        description="Perform basic arithmetic operations such as addition, subtraction, multiplication, and division.",
        when_to_use="Use this tool when you need to perform basic arithmetic operations.",
        inputs=[
            ToolParam(name="num1", type="float", description="The first number.", required=True),
            ToolParam(name="num2", type="float", description="The second number.", required=True),
            ToolParam(name="operation", type="string", description="The operation to perform: 'add', 'subtract', 'multiply', or 'divide'.", required=True)
        ],
        returns=ToolReturn(type="float", description="The result of the arithmetic operation.")
    )

    def __init__(self, final_answer: bool = True):
        self.final_answer = final_answer

    def __call__(self, num1: float, num2: float, operation: str) -> ToolResponse:
        if operation not in ["add", "subtract", "multiply", "divide"]:
            return ToolResponse(
                final_answer=False,
                observation="Invalid operation. Supported operations are 'add', 'subtract', 'multiply', and 'divide'.",
                extras={}
            )
        
        if operation == "divide" and num2 == 0:
            return ToolResponse(
                final_answer=False,
                observation="Division by zero is not allowed.",
                extras={}
            )
        try:
            num1 = float(num1)
            num2 = float(num2)
            if operation == "add":
                result = num1 + num2
            elif operation == "subtract":
                result = num1 - num2
            elif operation == "multiply":
                result = num1 * num2
            elif operation == "divide":
                result = num1 / num2
        except Exception as e:
            return ToolResponse(
                final_answer=False,
                observation=f"An error occurred: {str(e)}",
                extras={}
            )
        
        return ToolResponse(
            final_answer=self.final_answer,
            observation=float(result),
            extras={}
        )

toolkit = Toolkit(
    tools=[
        CalculatorTool(),
        GeneralQueryAssistantTool(llm=LLM, stream=True)
    ]
)

tools_router_agent = ToolsRouterAgent(
    toolkit=toolkit,
    llm=LLM,
    verbose=True,
    specific_instructions=""
)


INFO     | 2025-11-05 11:47:57 | toolkit.py:register_tool | Registered tool: calculator
INFO     | 2025-11-05 11:47:57 | toolkit.py:register_tool | Registered tool: general_query_assistant


Test ToolsRouterAgent

In [ ]:
query = "Perform the calculation 20 * 90"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")


INFO     | 2025-11-05 11:47:58 | tools_router_agent.py:run | [router] Using tool: calculator
INFO     | 2025-11-05 11:47:58 | tools_router_agent.py:run | [router] Raw inputs: {'num1': 20.0, 'num2': 90.0, 'operation': 'multiply'}
1800.0

In [36]:
query = "Tell me a light-hearted joke!"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")

INFO     | 2025-11-05 11:48:23 | tools_router_agent.py:run | [router] Using tool: general_query_assistant
INFO     | 2025-11-05 11:48:23 | tools_router_agent.py:run | [router] Raw inputs: {'query': 'Tell me a light-hearted joke!'}
Why don't skeletons fight each other? They don't have the guts!None

## ReactAgent (Agent bootstrap (Planner/Coder/Tester as ReActAgent))

In [38]:
from neurosurfer.agents.react import ReActAgent, ReActConfig
from neurosurfer.tools import Toolkit
# Tools
from tools.codegen import CodegenTool
from tools.fs_write import WriteFileTool
from tools.pytest_runner import PytestRunTool
from tools.lint import LintTool
from tools.docs_update import DocsUpdateTool
from tools.git_ops import GitOpsTool

SYSTEM_PROMPT = """
You are the Neurosurf Self-Improvement Agent.
Goals:
1) Plan changes to extend Neurosurf (tools/agents/adapters/docs/tests) safely.
2) Use only the registered tools. Validate with lint + pytest.
3) Produce small, reviewable increments. Write docs and tests for each change.
Rules:
- Never write outside the repository.
- Prefer minimal public APIs and follow existing code style.
- After green tests and lint, prepare a commit and short changelog.
"""

def build_toolkit() -> Toolkit:
    tk = Toolkit()
    # for t in [CodegenTool(), WriteFileTool(), PytestRunTool(), LintTool(), DocsUpdateTool(), GitOpsTool()]:
    for t in [CodegenTool(llm=LLM, rag=rag_agent), LintTool()]:
        tk.register_tool(t)
    return tk

def make_agent():
    # llm = OpenAIModel(model_name="gpt-4o-mini")  # or TransformersModel(...)
    # agent = ReActAgent(toolkit=build_toolkit(), llm=LLM, verbose=False, specific_instructions=SYSTEM_PROMPT)
    agent = ReActAgent(
        toolkit=build_toolkit(),
        llm=LLM,
        specific_instructions="Always be concise in your answers.",
        config=ReActConfig(
            temperature=0.6,
            max_new_tokens=4096,
            allow_input_pruning=True,
            repair_with_llm=True,
            skip_special_tokens=True
        )
    )
    return agent


agent = make_agent()
# Example task: scaffold a tool, write tests, run lint/tests, and prepare commit
# TASK = """Add a 'calculator' tool under neurosurf/tools/builtin/calculator.py, 
# write tests in tests/tools/test_calculator.py, add docs in docs/tools/calculator.md,
# ensure ruff/pytest pass, and commit on branch feature/calculator.
# """
TASK = """Generate a tool scientafic calculate."""

for chunk in agent.run(TASK, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")


INFO     | 2025-11-05 11:50:18 | toolkit.py:register_tool | Registered tool: codegen_tool
INFO     | 2025-11-05 11:50:18 | toolkit.py:register_tool | Registered tool: lint


[🧠] Chain of Thoughts...
Thought: I need to generate a tool named "scientific calculate" using the codegen_tool. I will provide the necessary inputs for the tool generation.

Action: {
  "tool": "codegen_tool",
  "inputs": {
    "name": "scientific_calculate",
    "module_path": "tools/scientific_calculate.py",
    "purpose": "Perform scientific calculations such as arithmetic operations, trigonometric functions, logarithms, and exponents.",
    "when_to_use": "Use this tool when you need to perform complex scientific calculations.",
    "inputs": [
      {"name": "expression", "type": "str", "description": "The mathematical expression to evaluate.", "required": true}
    ],
    "returns": {"type": "float", "description": "The result of the evaluated expression."}
  },
  "final_answer": true
}

[🔧] Tool: codegen_tool
[📤] Inputs: {'name': 'scientific_calculate', 'module_path': 'tools/scientific_calculate.py', 'purpose': 'Perform 
scientific calculations', 'when_to_use': 'When you need to perform complex scientific calculations', 'inputs': 
[{'name': 'expression', 'type': 'str', 'description': 'Mathematical expression to evaluate', 'required': True}], 
'returns': {'type': 'float', 'description': 'Result of the calculation'}}

Batches: 100%|██████████| 1/1 [00:00<00:00, 43.26it/s]

INFO     | 2025-11-05 11:50:27 | agent.py:retrieve | [RAGRetriever] Retrieved 6 chunks


Observation: Generated code has syntax error: invalid syntax (<unknown>, line 1)

...

[🔧] Tool: codegen_tool
[📤] Inputs: {'name': 'scientific_calculate', 'module_path': 'tools/scientific_calculate.py', 'purpose': 'Perform 
scientific calculations such as arithmetic operations, trigonometric functions, logarithms, and exponents.', 
'when_to_use': 'Use this tool when you need to perform complex scientific calculations.', 'inputs': [{'name': 
'expression', 'type': 'str', 'description': 'The mathematical expression to evaluate.', 'required': True}], 
'returns': {'type': 'float', 'description': 'The result of the evaluated expression.'}}

Batches: 100%|██████████| 1/1 [00:00<00:00, 43.34it/s]

INFO     | 2025-11-05 11:50:48 | agent.py:retrieve | [RAGRetriever] Retrieved 6 chunks


Observation: Generated and wrote: tools/scientific_calculate.py



[🧠] Chain of Thoughts...
Final Answer: The tool "scientific_calculate" has been generated and written to "tools/scientific_calculate.py". Please check the file for any potential syntax issues or further adjustments.